<a href="https://colab.research.google.com/github/choiyoonhyeok/MWFormer/blob/main/MWF_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install wandb

In [ ]:
import wandb
# 로그인(최초 1회만 브라우저 인증). 이미 로그인돼 있으면 건너뜀.
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: choiyoonhyeok0427 (choiyh0427-medical-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

GPU/파이썬 환경 확인

In [ ]:
!nvidia-smi

Mon Aug 25 18:01:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   45C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch, platform, sys
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA L4


레포 클론

In [ ]:
# 깔끔한 작업 디렉토리
%cd /content

# 본인 fork 레포 클론
!git clone https://github.com/choiyoonhyeok/MWFormer.git
%cd MWFormer
!git rev-parse --short HEAD

/content
Cloning into 'MWFormer'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (286/286), done.
remote: Compressing objects: 100% (266/266), done.
remote: Total 286 (delta 29), reused 265 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (286/286), 24.33 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/MWFormer
5128e95


필수 라이브러리 설치

In [ ]:
# requirements.txt 있으면 설치
!if [ -f requirements.txt ]; then \
    echo "requirements.txt detected → installing..."; \
    pip install -q -r requirements.txt; \
  else \
    echo "requirements.txt not found → skip"; \
  fi

# 공통 의존성 (누락 대비)
!pip install -q opencv-python einops timm yacs tqdm thop tensorboard albumentations

requirements.txt not found → skip


Google Drive 경로 변수 등록 & 존재 여부 점검

In [ ]:
from pathlib import Path

# === 경로 설정 ===
DRIVE_ROOT = Path("/content/drive/MyDrive/BOAZminiproject1")

DATA_ROOT = DRIVE_ROOT / "fulldata"  # rain/fog/dust 포함
WEIGHT_BACKBONE = DRIVE_ROOT / "MWFormer_weights" / "MWFormer_L" / "backbone.pth"
WEIGHT_STYLE    = DRIVE_ROOT / "MWFormer_weights" / "pretrained_feature_extraction_network" / "style_filter.pth"

print("DATA_ROOT        :", DATA_ROOT)
print("WEIGHT_BACKBONE  :", WEIGHT_BACKBONE)
print("WEIGHT_STYLE     :", WEIGHT_STYLE)

# === 존재 여부 점검 ===
assert DATA_ROOT.exists(), f"데이터 폴더가 없습니다: {DATA_ROOT}"
assert WEIGHT_BACKBONE.exists(), f"백본 가중치가 없습니다: {WEIGHT_BACKBONE}"
assert WEIGHT_STYLE.exists(),    f"style_filter 가중치가 없습니다: {WEIGHT_STYLE}"
print("✅ 모든 경로가 정상입니다.")

DATA_ROOT        : /content/drive/MyDrive/BOAZminiproject1/fulldata
WEIGHT_BACKBONE  : /content/drive/MyDrive/BOAZminiproject1/MWFormer_weights/MWFormer_L/backbone.pth
WEIGHT_STYLE     : /content/drive/MyDrive/BOAZminiproject1/MWFormer_weights/pretrained_feature_extraction_network/style_filter.pth
✅ 모든 경로가 정상입니다.


매칭 리스트 생성 스크립트 실행

3단계: 드라이런(sanity check) 학습

In [ ]:
# === MWFormer 패치 올인원 셀 ===
# - main_finetune.py
#   (1) 체크포인트 유연 로더(_load_state_dict_flexible) 삽입
#   (2) .module.state_dict() 사용부 치환
#   (3) TrainingDataset 타입 0/1/2 → 3으로 통일
#   (4) W&B 로깅: wandb.init / wandb.log(loss/lr/iter, initial PSNR) / wandb.finish()
#      * AMP(GradScaler) 사용 중이면 scaler.update() 뒤에 로그, 아니면 opts.step() 뒤에 로그
# - train_data_functions.py
#   (5) 'is not 3' → '!= 3'
#   (6) 리스트 파서: '두 칼럼 input  GT' 또는 '한 칼럼 input' 모두 지원
#       + train_data_dir 끝에 '/' 강제 + length 출력
# - utils_network.py
#   (7) SSIM/PSNR에 data_range=1.0 지정(중복 방지 포함)

from pathlib import Path
import re

# ---------- 1) main_finetune.py 패치 ----------
mf = Path("/content/MWFormer/main_finetune.py")
msrc = mf.read_text()

# (1) 유연 로더 헬퍼 삽입 (import torch 바로 뒤)
if "_load_state_dict_flexible" not in msrc:
    helper = r'''
from collections import OrderedDict

def _load_state_dict_flexible(ckpt_path, map_location="cpu"):
    obj = torch.load(ckpt_path, map_location=map_location)
    if hasattr(obj, "state_dict") and callable(getattr(obj, "state_dict")):
        sd = obj.state_dict()
    elif isinstance(obj, dict) and "state_dict" in obj and isinstance(obj["state_dict"], (dict, OrderedDict)):
        sd = obj["state_dict"]
    elif isinstance(obj, dict) and "model" in obj and isinstance(obj["model"], (dict, OrderedDict)):
        sd = obj["model"]
    elif isinstance(obj, (dict, OrderedDict)):
        sd = obj
    else:
        try:
            sd = obj.module.state_dict()
        except Exception:
            sd = obj
    new_sd = OrderedDict()
    for k, v in sd.items():
        nk = k[7:] if k.startswith("module.") else k
        new_sd[nk] = v
    return new_sd
'''
    msrc = re.sub(r"(import\s+torch[^\n]*\n)", r"\1" + helper + "\n", msrc, count=1)

# (2) .module.state_dict() → _load_state_dict_flexible(...)
msrc = re.sub(
    r"torch\.load\(([^)]*)\)\.module\.state_dict\(\)",
    r"_load_state_dict_flexible(\1)",
    msrc
)

# (3) Dataset 타입 통일: 0/1/2 → 3
msrc = msrc.replace("TrainingDataset(0, ", "TrainingDataset(3, ")
msrc = msrc.replace("TrainingDataset(1, ", "TrainingDataset(3, ")
msrc = msrc.replace("TrainingDataset(2, ", "TrainingDataset(3, ")

# (4-1) W&B import
if "import wandb" not in msrc:
    msrc = msrc.replace("from datetime import datetime", "from datetime import datetime\nimport wandb")

# (4-2) run_name 직후 wandb.init + LOG_EVERY
if "wandb.init(" not in msrc:
    msrc = msrc.replace(
        "run_name = f'{args.file_name}-{now}'",
        "run_name = f'{args.file_name}-{now}'\n"
        "    # === W&B init ===\n"
        "    wandb.init(project='MWFormer', name=run_name,\n"
        "               config={\n"
        "                   'file_name': args.file_name,\n"
        "                   'num_steps': args.num_steps,\n"
        "                   'learning_rate': args.learning_rate,\n"
        "                   'crop_size': args.crop_size,\n"
        "                   'num_workers': args.num_workers,\n"
        "                   'snapshot_dir': args.snapshot_dir,\n"
        "               })\n"
        "    LOG_EVERY = 100  # wandb 로그 주기(steps)\n"
    )

# (4-3) 초기 PSNR wandb.log
if "val_psnr_initial" not in msrc:
    msrc = msrc.replace(
        "print('initial model PSNR: ', old_val_psnr)",
        "print('initial model PSNR: ', old_val_psnr)\n"
        "    try:\n"
        "        wandb.log({'val_psnr_initial': float(old_val_psnr)}, step=0)\n"
        "    except Exception:\n"
        "        pass"
    )

# (4-4) 학습 루프 내 wandb.log(loss/lr/iter)
# AMP가 적용되어 있으면 scaler.update() 뒤에, 없으면 opts.step() 뒤에 삽입
if "scaler.update()" in msrc and "wandb.log({" not in msrc:
    msrc = msrc.replace(
        "scaler.update()",
        "scaler.update()\n"
        "        # === W&B log ===\n"
        "        try:\n"
        "            if (i_iter % LOG_EVERY) == 0:\n"
        "                wandb.log({'loss': float(loss_p.detach().item()), 'lr': float(main_lr), 'iter': int(i_iter)})\n"
        "        except Exception:\n"
        "            pass"
    )
elif "wandb.log({" not in msrc and "opts.step()" in msrc:
    msrc = msrc.replace(
        "opts.step()",
        "opts.step()\n"
        "        # === W&B log ===\n"
        "        try:\n"
        "            if (i_iter % LOG_EVERY) == 0:\n"
        "                wandb.log({'loss': float(loss_p.detach().item()), 'lr': float(main_lr), 'iter': int(i_iter)})\n"
        "        except Exception:\n"
        "            pass"
    )

# (4-5) 종료 시 wandb.finish()
if "wandb.finish()" not in msrc:
    msrc += "\ntry:\n    pass\nfinally:\n    import wandb as _w; _w.finish()\n"

mf.write_text(msrc)
print("✅ main_finetune.py 패치 완료 (유연 로더 + 타입 통일 + W&B 로깅)")

# ---------- 2) train_data_functions.py 패치 ----------
tp = Path("/content/MWFormer/train_data_functions.py")
tsrc = tp.read_text()

# (5) 'is not 3' 경고 수정
tsrc = tsrc.replace(" is not 3", " != 3")

# (6) 리스트 파서 교체 (with open(train_list) ~ self.train_data_dir 설정까지)
lines = tsrc.splitlines()
start_idx = None
end_idx = None
for i, ln in enumerate(lines):
    if "with open(train_list) as f:" in ln and start_idx is None:
        start_idx = i
    if start_idx is not None and "self.train_data_dir = train_data_dir" in ln:
        end_idx = i
        break

if start_idx is not None and end_idx is not None:
    new_block = [
        "        with open(train_list) as f:",
        "            contents = [ln.strip() for ln in f if ln.strip()]",
        "",
        "        input_names = []",
        "        gt_names    = []",
        "",
        "        for ln in contents:",
        "            parts = ln.split()  # 공백/탭 분리",
        "            if len(parts) >= 2:",
        "                # 두 칼럼: 0=입력, 1=GT",
        "                inp_rel = parts[0].lstrip('/')",
        "                gt_rel  = parts[1].lstrip('/')",
        "                input_names.append(inp_rel)",
        "                gt_names.append(gt_rel)",
        "            else:",
        "                # 한 칼럼: 입력만 → GT는 input→gt 치환",
        "                inp_rel = parts[0].lstrip('/')",
        "                gt_rel  = inp_rel.replace('/input/', '/gt/').replace('/GT/', '/gt/')",
        "                input_names.append(inp_rel)",
        "                gt_names.append(gt_rel)",
        "",
        "        self.input_names = input_names",
        "        self.gt_names    = gt_names",
        "        self.crop_size   = crop_size",
        "        self.train_data_dir = train_data_dir if train_data_dir.endswith('/') else (train_data_dir + '/')",
        "",
        "        print('length of input names:', len(input_names))",
    ]
    tsrc = "\n".join(lines[:start_idx] + new_block + lines[end_idx+1:])
else:
    print("⚠️ 리스트 파서 블록을 찾지 못했습니다. 파일 구조가 달라졌을 수 있습니다. (패치 건너뜀)")

tp.write_text(tsrc)
print("✅ train_data_functions.py 패치 완료 (리스트 파서 유연화)")

# ---------- 3) utils_network.py 패치 ----------
up = Path("/content/MWFormer/utils_network.py")
usrc = up.read_text()

# compare_ssim/compare_psnr → data_range=1.0 지정
usrc = re.sub(r"compare_ssim\(([^)]*)\)", r"compare_ssim(\1, data_range=1.0)", usrc)
usrc = re.sub(r"compare_psnr\(([^)]*)\)", r"compare_psnr(\1, data_range=1.0)", usrc)

# 혹시 이전 중복 인자가 남아있다면 정리
usrc = usrc.replace("data_range=1, data_range=1.0", "data_range=1.0")
usrc = usrc.replace("data_range=1.0, data_range=1.0", "data_range=1.0")

up.write_text(usrc)
print("✅ utils_network.py 패치 완료 (SSIM/PSNR data_range=1.0)")

print("\n🎉 모든 패치 완료! 이제 W&B 로깅과 유연한 데이터 로딩이 활성화되었습니다.")

✅ main_finetune.py 패치 완료 (유연 로더 + 타입 통일 + W&B 로깅)
✅ train_data_functions.py 패치 완료 (리스트 파서 유연화)
✅ utils_network.py 패치 완료 (SSIM/PSNR data_range=1.0)

🎉 모든 패치 완료! 이제 W&B 로깅과 유연한 데이터 로딩이 활성화되었습니다.


In [ ]:
from pathlib import Path

BASE = Path("/content/drive/MyDrive/BOAZminiproject1")
SRC  = BASE / "finetunning" / "train_pairs_joint.txt"  # 전체 리스트 (두 칼럼)
DST_DIR = BASE / "fulldata" / "lists"
DST_DIR.mkdir(parents=True, exist_ok=True)
DST  = DST_DIR / "train_pairs_joint.small.txt"

# 앞 800줄만 복사 (이미 있으면 건너뜀)
if SRC.exists() and not DST.exists():
    n = 800
    with open(SRC) as f, open(DST, "w") as w:
        for i, line in enumerate(f):
            if i >= n: break
            w.write(line)
print("샘플 리스트 위치:", DST)

샘플 리스트 위치: /content/drive/MyDrive/BOAZminiproject1/fulldata/lists/train_pairs_joint.small.txt


In [ ]:
from pathlib import Path

up = Path("/content/MWFormer/utils_network.py")
src = up.read_text()

# 잘못 들어간 중복 제거
src = src.replace("data_range=1, data_range=1.0", "data_range=1.0")
src = src.replace("data_range=1.0, data_range=1.0", "data_range=1.0")

# 혹시 남아있을 중복 대비: "data_range=1," -> "data_range=1.0,"
src = src.replace("data_range=1,", "data_range=1.0,")

up.write_text(src)
print("✅ utils_network.py 중복 data_range 인자 제거 완료")

✅ utils_network.py 중복 data_range 인자 제거 완료


In [ ]:
# ====== 로컬 캐시 + 진행률/ETA + 마일스톤(20/40/60/80/100%) 자동 백업 ======
from pathlib import Path
import shutil, os, time, math, json, subprocess, datetime

# === 경로 설정 ===
DRIVE      = Path("/content/drive/MyDrive/BOAZminiproject1")
SRC_ROOT   = DRIVE / "fulldata"
SRC_LIST   = SRC_ROOT / "lists" / "train_pairs_joint.txt"   # 두 칼럼 (input  gt)

DST_ROOT   = Path("/content/fastdata")                      # 캐시 타깃(로컬 SSD)
DST_LIST   = DST_ROOT / "lists" / "train_pairs_joint.txt"   # 성공 쌍만 다시 기록

# === 드라이브 백업 경로 (미러) ===
BACKUP_DIR = DRIVE / "fastdata_cache"                       # 미러 폴더(덮어쓰기/증분)
BACKUP_TAG_DIR = DRIVE / "fastdata_cache_milestones"        # 선택: 스냅샷(옵션)

# === 캐시/로그 옵션 ===
REPORT_EVERY      = 1000   # n쌍마다 로그
REPORT_EVERY_SEC  = 10     # 또는 n초마다 로그
RETRIES           = 6      # I/O 재시도
BASE_DELAY        = 0.4    # 재시도 지연(지수 증가)
MILESTONES        = [20, 40, 60, 80, 100]   # % 도달 시 백업
MAKE_SNAPSHOT_COPY = False  # True면 각 마일스톤별 별도 스냅샷 폴더도 생성(용량 주의)

# ====== Helper ======
def safe_exists(p: Path, retries=RETRIES, base_delay=BASE_DELAY):
    p = Path(p)
    for i in range(retries):
        try:
            return p.exists()
        except OSError:
            time.sleep(base_delay * (i + 1))
    return False

def safe_getsize(p: Path, retries=RETRIES, base_delay=BASE_DELAY):
    p = Path(p)
    for i in range(retries):
        try:
            return os.path.getsize(p)
        except OSError:
            time.sleep(base_delay * (i + 1))
    return None

def same_size(src: Path, dst: Path):
    s = safe_getsize(src)
    d = safe_getsize(dst)
    return (s is not None) and (d is not None) and (s == d)

def copy2_with_retry(src: Path, dst: Path, retries=RETRIES, base_delay=BASE_DELAY, verify=True):
    src = Path(src); dst = Path(dst)
    dst.parent.mkdir(parents=True, exist_ok=True)
    last_err = None
    for i in range(retries):
        try:
            shutil.copy2(src, dst)
            if not verify or same_size(src, dst):
                return True
        except OSError as e:
            last_err = e
            time.sleep(base_delay * (i + 1))
    # 마지막 검증
    if verify and dst.exists() and same_size(src, dst):
        return True
    if last_err:
        raise last_err
    return False

def fmt_time(sec):
    if sec is None or math.isinf(sec) or sec < 0:
        return "--:--:--"
    h = int(sec // 3600); m = int((sec % 3600) // 60); s = int(sec % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

def write_manifest(path: Path, payload: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w") as f:
        json.dump(payload, f, indent=2, ensure_ascii=False)

def rsync_available():
    try:
        out = subprocess.run(["rsync", "--version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return out.returncode == 0
    except FileNotFoundError:
        return False

def backup_to_drive(src_dir: Path, dst_dir: Path, milestone_pct: int):
    """
    /content/fastdata → Drive로 증분 백업(가능하면 rsync 사용).
    milestone_pct: 진행률(정수)
    """
    src_dir = Path(src_dir); dst_dir = Path(dst_dir)
    dst_dir.mkdir(parents=True, exist_ok=True)

    start = time.time()
    print(f"🔄 백업 시작({milestone_pct}%): {src_dir} → {dst_dir}")

    # 1) rsync가 있으면 rsync로 빠르게 동기화(증분)
    if rsync_available():
        # -a (archive), -r (recursive), -t (times), --delete(삭제 동기화), --update(새로운/수정된 파일만)
        cmd = [
            "rsync", "-a", "--delete", "--update",
            f"{str(src_dir).rstrip('/')}/", f"{str(dst_dir).rstrip('/')}/"
        ]
        proc = subprocess.run(cmd)
        if proc.returncode != 0:
            print("⚠️ rsync 실패 → shutil로 폴백합니다.")
            _copytree_update(src_dir, dst_dir)
    else:
        # 2) rsync가 없으면 느리지만 파이썬으로 증분 복사
        _copytree_update(src_dir, dst_dir)

    # 마일스톤 마커/메타 저장
    marker = dst_dir / f"_milestone_{milestone_pct}p.txt"
    marker.write_text(f"milestone: {milestone_pct}%\nfinished_at: {datetime.datetime.now().isoformat()}")

    meta = {
        "milestone_pct": milestone_pct,
        "finished_at": datetime.datetime.now().isoformat()
    }
    write_manifest(dst_dir / "cache_backup_meta.json", meta)

    # (옵션) 스냅샷 폴더 별도 생성
    if MAKE_SNAPSHOT_COPY:
        SNAP = Path(BACKUP_TAG_DIR) / f"milestone_{milestone_pct:03d}p"
        print(f"📦 스냅샷 저장: {SNAP}")
        if rsync_available():
            SNAP.parent.mkdir(parents=True, exist_ok=True)
            cmd2 = ["rsync", "-a", "--delete", "--update",
                    f"{str(dst_dir).rstrip('/')}/", f"{str(SNAP).rstrip('/')}/"]
            subprocess.run(cmd2)
        else:
            _copytree_update(dst_dir, SNAP)

    took = fmt_time(time.time() - start)
    print(f"✅ 백업 완료({milestone_pct}%) | 소요: {took}")

def _copytree_update(src_root: Path, dst_root: Path):
    """dst에 같은 경로/이름이 있고 사이즈 같으면 스킵, 아니면 업데이트 복사"""
    src_root = Path(src_root); dst_root = Path(dst_root)
    for root, dirs, files in os.walk(src_root):
        rel = os.path.relpath(root, src_root)
        dst_dir = os.path.join(dst_root, rel) if rel != "." else str(dst_root)
        os.makedirs(dst_dir, exist_ok=True)
        for fn in files:
            s = Path(root) / fn
            d = Path(dst_dir) / fn
            try:
                if d.exists() and same_size(s, d):
                    continue
                shutil.copy2(s, d)
            except Exception:
                # 파일 단위 에러는 무시하고 계속 진행(대부분 일시적 I/O)
                pass

# ====== 리스트 읽기 ======
assert safe_exists(SRC_LIST), f"리스트가 보이지 않습니다: {SRC_LIST}\n드라이브 마운트를 확인하세요."
with open(SRC_LIST) as f:
    raw_lines = [ln.strip() for ln in f if ln.strip()]

pairs_all = []
for ln in raw_lines:
    parts = ln.split()
    if len(parts) >= 2:
        pairs_all.append((parts[0].lstrip("/"), parts[1].lstrip("/")))
TOTAL = len(pairs_all)
print(f"총 쌍 수: {TOTAL:,}")

# ====== 캐시 루프 ======
ok_pairs, copied_files, skipped_files, missing, failed = 0, 0, 0, 0, 0
pairs_ok_list = []

start = time.time()
last_report_t = start
# 마일스톤 관리(한 번만 수행)
pending_milestones = set(MILESTONES)

# 미리 폴더 생성
(DST_ROOT / "lists").mkdir(parents=True, exist_ok=True)
BACKUP_DIR.mkdir(parents=True, exist_ok=True)
BACKUP_TAG_DIR.mkdir(parents=True, exist_ok=True)

for idx, (inp_rel, gt_rel) in enumerate(pairs_all, 1):
    s_inp, s_gt = SRC_ROOT / inp_rel, SRC_ROOT / gt_rel
    d_inp, d_gt = DST_ROOT / inp_rel, DST_ROOT / gt_rel
    d_inp.parent.mkdir(parents=True, exist_ok=True)
    d_gt.parent.mkdir(parents=True, exist_ok=True)

    ok = True

    # input
    if safe_exists(s_inp):
        if safe_exists(d_inp) and same_size(s_inp, d_inp):
            skipped_files += 1
        else:
            try:
                copy2_with_retry(s_inp, d_inp)
                copied_files += 1
            except OSError:
                ok = False
                failed += 1
    else:
        ok = False
        missing += 1

    # gt
    if safe_exists(s_gt):
        if safe_exists(d_gt) and same_size(s_gt, d_gt):
            skipped_files += 1
        else:
            try:
                copy2_with_retry(s_gt, d_gt)
                copied_files += 1
            except OSError:
                ok = False
                failed += 1
    else:
        ok = False
        missing += 1

    if ok:
        ok_pairs += 1
        pairs_ok_list.append(f"{inp_rel} {gt_rel}")

    # 진행률/ETA 출력
    now = time.time()
    need_time_report = (idx % REPORT_EVERY == 0) or (now - last_report_t >= REPORT_EVERY_SEC)
    if need_time_report:
        elapsed = now - start
        pct_idx = idx / TOTAL * 100 if TOTAL else 0.0
        pct_ok  = ok_pairs / TOTAL * 100 if TOTAL else 0.0
        sec_per_pair = elapsed / idx if idx else float("inf")
        eta_sec = sec_per_pair * (TOTAL - idx)
        print(
            f"[{idx:,}/{TOTAL:,}] "
            f"done={ok_pairs:,} ({pct_ok:5.1f}%)  "
            f"progress={pct_idx:5.1f}%  "
            f"copied={copied_files:,}  skipped={skipped_files:,}  "
            f"missing={missing:,}  failed={failed:,}  "
            f"elapsed={fmt_time(elapsed)}  ETA={fmt_time(eta_sec)}"
        )
        last_report_t = now

    # === 마일스톤 도달 시 자동 백업 ===
    done_pct_int = int(ok_pairs / TOTAL * 100) if TOTAL else 0
    hit = sorted([m for m in pending_milestones if done_pct_int >= m])
    if hit:
        # 중복 방지: 가장 낮은 마일스톤부터 순서대로 실행 후 pending에서 제거
        for m in hit:
            try:
                backup_to_drive(DST_ROOT, BACKUP_DIR, m)
            except Exception as e:
                print(f"⚠️ 마일스톤 {m}% 백업 실패: {e}")
            pending_milestones.discard(m)

# 리스트 저장(성공 쌍만)
DST_LIST.parent.mkdir(parents=True, exist_ok=True)
with open(DST_LIST, "w") as w:
    w.write("\n".join(pairs_ok_list))

elapsed = time.time() - start
print("\n✅ 로컬 캐시 완료")
print(f"쌍(OK)   : {ok_pairs:,} / {TOTAL:,}")
print(f"복사(파일): {copied_files:,}  | 스킵: {skipped_files:,}  | 누락: {missing:,}  | 실패: {failed:,}")
print(f"경과시간  : {fmt_time(elapsed)}")
print("로컬 리스트:", DST_LIST)

# 최종 마니페스트 + 마지막 백업(혹시 100% 마일스톤을 못 탔다면 한 번 더)
manifest = {
    "ok_pairs": ok_pairs,
    "total_pairs": TOTAL,
    "elapsed": elapsed,
    "finished_at": datetime.datetime.now().isoformat(),
}
write_manifest(DST_ROOT / "cache_progress.json", manifest)

if 100 in pending_milestones:
    try:
        backup_to_drive(DST_ROOT, BACKUP_DIR, 100)
    except Exception as e:
        print(f"⚠️ 최종 백업 실패: {e}")

print("🎉 캐싱 & 마일스톤 백업 종료")

AssertionError: 리스트가 보이지 않습니다: /content/drive/MyDrive/BOAZminiproject1/fulldata/lists/train_pairs_joint.txt
드라이브 마운트를 확인하세요.

##캐싱 끝나면 무조건 드라이브에 넣을 것!!!!

In [ ]:
# 수동 드라이브 저장
# !cp -r /content/fastdata /content/drive/MyDrive/BOAZminiproject1/fastdata_cache

In [ ]:
# 나중에 불러오기
!cp -r /content/drive/MyDrive/BOAZminiproject1/fastdata_cache /content/fastdata

최소 패치(AMP + 작은 배치 + 드라이브 I/O 재시도)

In [ ]:
# === 최소 패치: main_finetune(AMP/작은배치) + train_data_functions(open_with_retry) ===
from pathlib import Path, re as _re

# ---------- main_finetune.py ----------
mp = Path("/content/MWFormer/main_finetune.py")
msrc = mp.read_text()

# (A) AMP import (중복 방지)
if "from torch.cuda.amp import autocast, GradScaler" not in msrc:
    msrc = msrc.replace(
        "from datetime import datetime",
        "from datetime import datetime\nfrom torch.cuda.amp import autocast, GradScaler"
    )

# (B) scaler 초기화 (중복 방지)
if "scaler = GradScaler()" not in msrc:
    msrc = msrc.replace(
        "model.train()",
        "model.train()\n\n    # AMP scaler (메모리 절약)\n    scaler = GradScaler()"
    )

# (C) 학습 루프의 backward/step을 AMP로 감싸기 (패턴이 다르면 건너뜀)
if "scaler.scale(" not in msrc:
    msrc = _re.sub(
        r"opts\.zero_grad\(\)\s*\n([\s\S]*?)loss_p\s*=\s*loss_calc\((.*?)\)\s*\n\s*loss_p\.backward\(\)\s*\n\s*opts\.step\(\)",
        (
            "opts.zero_grad()\n"
            "\\1with autocast():\n"
            "            loss_p = loss_calc(\\2)\n"
            "        scaler.scale(loss_p).backward()\n"
            "        scaler.step(opts)\n"
            "        scaler.update()"
        ),
        msrc,
        count=1
    )

# (D) DataLoader 배치 크기 최소화(10/11/4 → 2), 여러 곳에 있을 수 있어 치환
msrc = msrc.replace("batch_size=10", "batch_size=2")
msrc = msrc.replace("batch_size=11", "batch_size=2")
msrc = msrc.replace("batch_size=4",  "batch_size=2")

mp.write_text(msrc)
print("✅ main_finetune.py: AMP/작은배치 패치 완료")

# ---------- train_data_functions.py ----------
tp = Path("/content/MWFormer/train_data_functions.py")
tsrc = tp.read_text()

# (E) open_with_retry 헬퍼 삽입(중복 방지)
if "def open_with_retry(" not in tsrc:
    helper = r"""

import time
from PIL import Image

def open_with_retry(img_path, max_retries=6, delay=0.4):
    last_err = None
    for _ in range(max_retries):
        try:
            with Image.open(img_path) as im:
                return im.convert('RGB')
        except Exception as e:
            last_err = e
            time.sleep(delay)
    raise last_err
"""
    tsrc = tsrc.replace("ImageFile.LOAD_TRUNCATED_IMAGES = True", "ImageFile.LOAD_TRUNCATED_IMAGES = True\n" + helper)

# (F) 실제 이미지 로딩 부분을 open_with_retry로 교체(존재 시에만)
tsrc = _re.sub(
    r"input_img = Image\.open\(self\.train_data_dir \+ input_name\)\s*?\n\s*try:\s*?\n\s*gt_img = Image\.open\(self\.train_data_dir \+ gt_name\)\s*?\n\s*except:\s*?\n\s*gt_img = Image\.open\(self\.train_data_dir \+ gt_name\)\.convert\('RGB'\)",
    "input_img = open_with_retry(self.train_data_dir + input_name)\n        gt_img    = open_with_retry(self.train_data_dir + gt_name)",
    tsrc
)

# (G) 경고 수정: 'is not 3' → '!= 3'
tsrc = tsrc.replace(" is not 3", " != 3")

tp.write_text(tsrc)
print("✅ train_data_functions.py: open_with_retry/경고수정 패치 완료")

경로/리스트 점검(빠르게)

In [ ]:
from pathlib import Path

FAST_ROOT = Path("/content/fastdata")
LIST_PATH = FAST_ROOT / "lists" / "train_pairs_joint.txt"

print("fastdata 존재:", FAST_ROOT.exists())
print("train_pairs_joint.txt 존재:", LIST_PATH.exists())
if LIST_PATH.exists():
    print("라인 수:", sum(1 for _ in open(LIST_PATH)))

학습 실행(캐시 사용, 안정 세팅)

In [ ]:
%cd /content/MWFormer

!python main_finetune.py \
  -train_data_dir /content/fastdata/ \
  -labeled_name lists/train_pairs_joint.txt \
  -restore-from-stylefilter /content/drive/MyDrive/BOAZminiproject1/MWFormer_weights/pretrained_feature_extraction_network/style_filter.pth \
  -restore-from /content/drive/MyDrive/BOAZminiproject1/MWFormer_weights/MWFormer_L/backbone.pth \
  -num-workers 2 \
  -crop_size 256 256 \
  -save-pred-every 10000 \
  -loss_save_step 10000 \
  -snapshot-dir /content/drive/MyDrive/BOAZminiproject1/finetunning/snapshots \
  -file-name MWFormer_L_wandb_fast